## 1. Env. setup

### 1.1 Role Check
The ```AmazonSageMaker-ExecutionRole-XXXXXXXXTXXXXXX``` has to have the following 2 policies
- `AmazonBedrockFullAccess`
- `AmazonRedshiftFullAccess`

In [ ]:
from sagemaker import get_execution_role

In [ ]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

### 1.2 Package Install
- After the installation is completed, the kernel will be killed.

In [ ]:
import os
import sys
import IPython
import subprocess

install_needed = True

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U awscli
    !{sys.executable} -m pip install -U botocore
    !{sys.executable} -m pip install -U boto3
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U langchain
    !{sys.executable} -m pip install -U langchain-community
    !{sys.executable} -m pip install -U langchain_aws
    !{sys.executable} -m pip install -U langchain-experimental
    !{sys.executable} -m pip install -U langgraph
    !{sys.executable} -m pip install -U sqlalchemy #==2.0.1
    !{sys.executable} -m pip install -U anthropic
    !{sys.executable} -m pip install -U python-dotenv

    response = subprocess.run(['cat', '/etc/os-release'], capture_output=True)
    response = response.stdout.decode("utf-8")
    IPython.Application.instance().kernel.do_shutdown(True)

## 2. Data Upload

In [ ]:
import os
oracle_sp_path = "./data/oracle_sp" # Oracle SP가 저장된 폴더 경로

if not os.path.exists(oracle_sp_path):
    os.makedirs(oracle_sp_path)

<span style="color:red"><strong>Please upload all your Oracle Stored Procedures as a text format under `./data/oracle_sp` folder before running the below cell.</strong></span>
```
.
├── data
│   └── oracle_sp
│       ├── oracle-stored-procedure-name-1.txt
│       ├── oracle-stored-procedure-name-2.txt
│       ├── oracle-stored-procedure-name-3.txt
|       ├── ...
│       └── oracle-stored-procedure-name-n.txt
```

In [ ]:
import os

def check_file_extension(file_path):
    # Get the file extension
    file_name, file_extension = os.path.splitext(file_path)
    # Check if the extension is .txt
    if file_extension.lower() == ".txt":
        return True
    else:
        print(f"{file_name} is not a text file.")
        return False

oracle_sp_list = [file for file in os.listdir(oracle_sp_path) if not file.startswith('.')]

if len(oracle_sp_list) == 0:
    print("Please upload Oracle's stored procedures into oralce_sp_path, or you will get an error.")
else:
    for oracle_sp in oracle_sp_list:
        if check_file_extension(oracle_sp) == False:
            print("Please upload Oracle's stored procedures as a text format, or you will get an error.")

## 3. LLM Load

In [ ]:
import os
import json
import boto3
import awscli
import botocore
import langchain
from langchain_aws import ChatBedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.embeddings import BedrockEmbeddings

In [ ]:
# Model ID
claude_v3_haiku = "anthropic.claude-3-haiku-20240307-v1:0"
claude_v3_sonnet = "anthropic.claude-3-sonnet-20240229-v1:0"
claude_v3_5_sonnet = "anthropic.claude-3-5-sonnet-20240620-v1:0"
claude_v3_opus = "anthropic.claude-3-sonnet-20240229-v1:0"

In [ ]:
bedrock_client = boto3.client(service_name="bedrock-runtime", region_name="us-west-2")

llm_init_converter = ChatBedrock(
    model_id=claude_v3_sonnet,
    client=bedrock_client,
    streaming=True,
    callbacks=[], #[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "max_tokens": 4096,
        "stop_sequences": ["</sql>"],
        "temperature": 0.9,
        # "top_k": 350,
        # "top_p": 0.999
    }
)

llm_retry_converter = ChatBedrock(
    model_id=claude_v3_sonnet,
    client=bedrock_client,
    streaming=True,
    callbacks=[], #[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "max_tokens": 4096,
        "stop_sequences": ["</sql>"],
        "temperature": 0.9,
        # "top_k": 350,
        # "top_p": 0.999
    }
)

## 4. Chain

In [ ]:
#from langchain.schema import Document
from langchain_core.messages import HumanMessage
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

### 4.1 Chain for Initial Conversion

In [ ]:
init_memory = ConversationBufferWindowMemory(k=3, output_key='response') #return_messages=True)
init_memory.clear()

init_system_prompt = '''
You are an assistant tasked with converting Oracle's stored procedure to Amazon Redshift's stored procedure.
The Oracle stored procedure will be given by human.

Amazon Redshift's stored procedure that you convert MUST follow the below structure:
<Amazon Redshift's stored procedure structure>
CREATE OR REPLACE PROCEDURE procedure_name( 
    parameter1 datatype, 
    parameter2 datatype, 
    ... 
)
LANGUAGE plpgsql 
AS $$ 
DECLARE 
-- Variable declarations 
variable1 datatype;
variable2 datatype; 
...

BEGIN 
-- Procedure logic 
-- SQL statements 
-- Control structures (IF, CASE, LOOP, etc.) 
-- Function calls 
-- Exception handling

EXCEPTION 
    WHEN exception_condition1 THEN 
        -- Exception handling logic 
    WHEN exception_condition2 THEN 
        -- Exception handling logic 
    ... 
    WHEN OTHERS THEN 
        -- General exception handling logic 
END; 
$$;
</Amazon Redshift's stored procedure structure>

When converting the stored procedure, you MUST follow the below instructions:
<instructions>
- Ensure that Oracle uses PL/SQL while Amazon Redshift uses PL/pgSQL.
- Do NOT use Oracle-specific functions and syntax in Redshift's stored procedure.
- **In Redshift, do NOT use GOTO statement; use IF-THEN-ELSE statement instead of GOTO statement.**
- In Redshift, do NOT use PERFORM statement; use CALL statement instead of PERFORM statement.
- In Redshift, do NOT use NOLOGGING statement.
- In Redshift, do NOT use number data type; use numeric data type instead of number data type.
- In Redshift, do NOT use varchar2 data type; use varchar data type instead of varchar2 data type.
- In Redshift, do NOT use built-in functions directly in RAISE; time-related logic should be assigned to variables and processed before outputting RAISE logs.
- In Redshift, CALL is used to invoke other stored procedure. For example: CALL SP_APPL_LOG(PV_MOD_NAME, 'START.. [' || IN_YM || ']');
- In Redshift, Nested BEGIN can be used.
- In Redshift, when converting cursor FOR loops, declare the loop variable as a RECORD type at the beginning of the procedure, and use this variable directly in the loop without additional DECLARE statements. For example: DECLARE r RECORD; BEGIN FOR r IN (SELECT * FROM table) LOOP -- Use r.column_name to access fields END LOOP; END;
- If the variable's data type is declared with %TYPE attribute, define the data type as VARCHAR and tag a WARNING comment.
- Preserve any comments starting with '--' exactly as they are in the given oracle stored procedure, including their spacing and location relative to the code elements.
- Only provide the converted AWS Redshift stored procedure code and comments. Do not include any extra text or explanations.
- Put the converted stored procedure in <sql></sql> tags.
- If chat history is provided, it indicates you already started converting an Oracle stored procedure into AWS Redshift stored procedure but stopped due to the max token limit. Continue EXACTLY from where it left off.
</instructions>

<history>
Here is a chat history so far: {history}
</history>
'''



init_prompt = ChatPromptTemplate.from_messages([
    ("system", init_system_prompt),
    ("human", "{input}"),
    #("ai", "Here is the converted stored procedure: <sql>"),
    ("ai", "<sql>")
])

init_chain = ConversationChain(
    memory=init_memory,
    prompt=init_prompt,
    input_key="input",
    llm=llm_init_converter,
    return_final_only=False,
    verbose=False,
)

In [ ]:
def init_converter(init_chain, oracle_sp):

    response = ""
    partial_response = init_chain.invoke(
        {
            "input":f"Here is an Oracle stored procedure: {oracle_sp}"
        }
    )
    
    while partial_response['full_generation'][0].message.response_metadata['stop_reason'] == "max_tokens":
        #response += partial_response['response'].split('<sql>')[1]
        
        response += partial_response['response'].replace("\n", "", 1)
        
        partial_response = init_chain.invoke(
            {
                "input": "<instruction>continue generation without overlapping a single character with what you've just generated.</instruction>"
            }
        )

    #response += partial_response['response'].split('<sql>')[1]
    response += partial_response['response'].replace("\n", "", 1)
    
    return response

### 4.2 Chain for Retry Conversion

In [ ]:
retry_memory = ConversationBufferWindowMemory(k=3, output_key='response') #return_messages=True)
retry_memory.clear()

retry_system_prompt = '''
You are an AI assistant tasked with correcting syntax error encountered when converting Oracle's stored procedure to AWS Redshift's stored procedure.
The converted AWS Redshift stored procedure, syntax error will be given by human.

When correcting Amazon Redshift stored procedure, you MUST follow the below instructions:
<instructions>
- Modify ONLY part of the stored procedure that cause syntax errors; do NOT modify other part.
- Preserve any comments starting with '--' exactly as they are in the given oracle stored procedure, including their spacing and location relative to the code elements.
- Put the converted stored procedure in <sql></sql> tags.
- Only provide the converted AWS Redshift stored procedure code and comments. Do not include any extra text or explanations.
- If chat history is provided, it indicates you already started correcting the AWS Redshift stored procedure but stopped due to the max token limit. Continue EXACTLY from where it left off.
</instructions>

Here's are useful information of common syntax errors and how to remediate them:
<reference>
If the syntax error includes 'TO_CHAR', check the following information:
The RAISE level statement has the following syntax: RAISE level 'format' [, variable [, ...]]; Inside the format string, % is replaced by the next optional argument's string representation. optional arguments must be simple variables, not expressions, and the format must be a simple string literal like the following example.
<example>
Oracle Stored Procedure: 
RAISE NOTICE 'START_TIME = %', TO_CHAR(CURRENT_TIMESTAMP, 'HH24:MI:SS');
RAISE INFO 'END_TIME   = %', TO_CHAR(CURRENT_TIMESTAMP, 'YYYY-MM-DD HH24:MI:SS');

Redshift Stored Procedure: 
DECLARE
    LV_START_TIME VARCHAR(20);
    LV_END_TIME VARCHAR(20);
BEGIN
    LV_START_TIME := TO_CHAR(CURRENT_TIMESTAMP, 'HH24:MI:SS.MS');
    RAISE INFO 'START_TIME = %', LV_START_TIME;
    LV_END_TIME := TO_CHAR(CURRENT_TIMESTAMP, 'HH24:MI:SS.MS');
    RAISE INFO 'END_TIME = %', LV_END_TIME;
END;
</example>

If the syntax error includes 'GOTO', check the following information:
In Redshift, do NOT use GOTO statement; use IF-THEN-ELSE statement instead of GOTO statement.

If the syntax error includes 'PERFORM', check the following information:
In Redshift, do NOT use PERFORM statement; use CALL statement instead of PERFORM statement.

If the syntax error includes invalid type name with %TYPE attribute, check the following information:
This error indicates that Redshift doesn't recognize the %TYPE attribute, which is commonly used in Oracle PL/SQL but not supported in Redshift's stored procedures. 
Declare the data type with VARCHAR for all variables that were using the %TYPE attribute and tag a WARNING comment as a walkaround.

If the syntax error includes 'LOOP', check the following information:
When converting cursor FOR loops, declare the loop variable as a RECORD type at the beginning of the procedure, and use this variable directly in the loop without additional DECLARE statements. 
<example>
DECLARE r RECORD; BEGIN FOR r IN (SELECT * FROM table) LOOP -- Use r.column_name to access fields END LOOP;
</example>


</reference>


<history>
Here is a chat history so far: {history}
</history>
'''

retry_prompt = ChatPromptTemplate.from_messages([
    ("system", retry_system_prompt),
    ("human", "{input}"),
    #("ai", "Here is the converted stored procedure: <sql>"),
    ("ai", "<sql>")
])

retry_chain = ConversationChain(
    memory=retry_memory,
    prompt=retry_prompt,
    input_key="input",
    llm=llm_retry_converter,
    return_final_only=False,
    verbose=False,
)

In [ ]:
def retry_converter(retry_chain, redshift_sp, syntax_error):

    response = ""
    
    retry_human_prompt = f'''
    Here's Amazon Redshift stored procedure: <Redshift stored procedure>{redshift_sp}</Redshift stored procedure>
    Here's syntax error: <sytax error>{syntax_error}</sytax error>
    '''
    
    partial_response = retry_chain.invoke(
        {
            "input": retry_human_prompt
        }
    )
    
    while partial_response['full_generation'][0].message.response_metadata['stop_reason'] == "max_tokens":
        response += partial_response['response'].replace("\n", "", 1)
        
        partial_response = retry_chain.invoke(
            {
                "input": "<instruction>continue generation without overlapping a single character with what you've just generated.</instruction>"
            }
        )
    
    response += partial_response['response'].replace("\n", "", 1)
    
    return response

## 5. Conversion

In [ ]:
succeeded_sp_path = './data/redshift_sp/success' # Path to the folder where the Redshift SP will be stored after conversion
failed_sp_path = './data/redshift_sp/fail' # Path to the folder where the Redshift SP will be stored after conversion

In [ ]:
# 0. Make folders to store the conversion and query resutls
if not os.path.exists(succeeded_sp_path):
    os.makedirs(succeeded_sp_path)
if not os.path.exists(failed_sp_path):
    os.makedirs(failed_sp_path)
if not os.path.exists(f'{failed_sp_path}/error_message'):
    os.makedirs(f'{failed_sp_path}/error_message')

In [ ]:
def final_converter(redshift_serverless_query_execute, oracle_sp_path, max_retry, succeeded_sp_path, failed_sp_path):

    #1. Initial Conversion
    print("### Starting Initial Conversion ###")
    oracle_sp_name = oracle_sp_path.split("/")[-1]
    with open(oracle_sp_path, 'r', encoding='utf-8') as file:
        oracle_sp = file.read()
        
    init_memory.clear()
    init_redshift_sp = init_converter(init_chain, oracle_sp)
    
    
    #2. Run the Initial Conversion at AWS Redshift and Check the result
    init_query_id = redshift_serverless_query_execute.execute_query(init_redshift_sp)
    status, query_result = redshift_serverless_query_execute.syntax_checker(init_query_id)
    
    #3. Retry Conversion based on Syntax Error Message from AWS Redshift
    current_retry = 0
    while current_retry < max_retry:
        if status == "FINISHED":
            break
        elif status == "FAILED":
            redshift_sp = init_redshift_sp if current_retry == 0 else retry_redshift_sp

            print(query_result)
            current_retry += 1
            
            print(f"### Starting Retry Conversion : # {current_retry} ###")
            retry_memory.clear()
            retry_redshift_sp = retry_converter(
                retry_chain, 
                redshift_sp, 
                query_result
            )
            
            retry_query_id = redshift_serverless_query_execute.execute_query(retry_redshift_sp)
            status, query_result = redshift_serverless_query_execute.syntax_checker(retry_query_id)

    #4. Store the conversion and query results
    if status == "FINISHED":
        # If a conversion is successfully done w/o syntax error, store the converted sp in ./data/redshift_sp/success/
        with open(f'{succeeded_sp_path}/{oracle_sp_name}', 'w', encoding='utf-8') as file:
            file.write(init_redshift_sp)
        print("FINISHED\n" + query_result)
        print("\n\n")
        
    elif status == "FAILED":
        # If a retry conversion is failed w/ syntax error, store the converted sp in ./data/redshift_sp/fail/ and syntax error message in ./data/redshift_sp/fail/error_message/
        with open(f'{failed_sp_path}/{oracle_sp_name}', 'w', encoding='utf-8') as file:
            file.write(retry_redshift_sp)
        with open(f'{failed_sp_path}/error_message/error-message-{oracle_sp_name}', 'w', encoding='utf-8') as file:
            file.write(query_result)
        print("FAILED\n" + query_result)
        print("\n\n")

<span style="color:red">Please set your Redshift Serverless Database name as ```REDSHIFT_DATABASE_NAME``` and Redshift Serverless Workgroup name as ```REDSHIFT_WORKGROUP_NAME``` in the below cell.</span>

In [ ]:
from redshift_execution import RedshiftServerlessQueryExecute

REDSHIFT_DATABASE_NAME="dev"
REDSHIFT_WORKGROUP_NAME="cdk-workgroup"

redshift_serverless_query_execute = RedshiftServerlessQueryExecute(
    redshift_database_name=REDSHIFT_DATABASE_NAME,
    redshift_workgroup_name=REDSHIFT_WORKGROUP_NAME
)

<span style="color:red">The below cell will attept to convert Oracle stored procedure to Redshift one <strong>by calling the Amazon Bedrock API and Redshift Data API, which incur costs.</strong></span>

In [ ]:
from tqdm.auto import tqdm
import time

max_retry = 2
oracle_sp_names = os.listdir(oracle_sp_path)
oracle_sp_names = [file for file in os.listdir(oracle_sp_path) if not file.startswith('.')]

for oracle_sp_name in tqdm(oracle_sp_names):
    each_oracle_sp_path = os.path.join(oracle_sp_path, oracle_sp_name)
    print(each_oracle_sp_path)
    try:
        final_converter(redshift_serverless_query_execute, each_oracle_sp_path, max_retry, succeeded_sp_path, failed_sp_path)
    except ThrottlingException:
        print("You exceed the maximum number of API requests that are allowed per second.\nThe conversion will restart after 30 seconds of sleep.")
        time.sleep(30)

## 6. Conversion Report

In [ ]:
import pandas as pd

oracle_sp_list = [file for file in os.listdir(oracle_sp_path) if not file.startswith('.')]
oracle_sp_name_list = [oracle_sp_name.split('.txt')[0] for oracle_sp_name in os.listdir(oracle_sp_path)]
redshift_sp_name_list = [redshift_sp_name.split('.txt')[0] for redshift_sp_name in os.listdir(succeeded_sp_path)]

results = []
for oracle_sp_name in oracle_sp_name_list:
    if oracle_sp_name in redshift_sp_name_list:
        results.append([oracle_sp_name, 1, "No Error"])
    else:
        try:
            with open(f'{failed_sp_path}/error_message/error-message-{oracle_sp_name}.txt', 'r', encoding='utf-8') as file:
                syntax_error = file.read()
            results.append([oracle_sp_name, 0, syntax_error])
        except Exception as e:
            print(e)
            results.append([oracle_sp_name, 0, "Unknown Exception"])
    
results_pd = pd.DataFrame(results, columns=["Stored Procedure Name", "Conversion Result", "Syntax Error"])
results_pd.to_csv('./conversion_report.csv', sep=',', na_rep='NaN')

In [ ]:
conversion_rate = results_pd['Conversion Result'].mean()
print(f"Conversion Rate : {conversion_rate:.2%}")